# Welcome to the neighbourhood-scale thermal comfort mapping with SOLWEIG

---

What to advise the local government of a city on the heat stress reduction benefits?
* Which parts of a neighbourhood are susceptible to thermal discomfort?
* What are the likely causes of that?
* How much heat stress could be alleviated by targeted interventions
* How should these interventions look like?



To answer these questions, we invite you to dive into the model results:
* Understand the neighbourhood, its context, and how it's represented in the model
* Explore the heatwave characteristics and urban patterns
* Make sense of the output, its patterns, and what it reveals

But ... don't feel limited by what we've provided!

Feel free to bring in your own tools, datasets, and creative approaches ...whatever you feel comfortable with or curious about. Hackathons are meant for exploration, not constraint.

Enjoy!
---

## Setting up what we need...

* Mount the Google Drive, to access files and datasets
* Install the required packages and tools
* Set up directory and filenames (**some need to be adjusted**)

In [ ]:
# Some Initial imports
import os, re, sys
import warnings
import importlib
from datetime import datetime
from glob import glob

# Ignore some warnings
warnings.filterwarnings("ignore", message=".*find_spec.*", category=ImportWarning)
for warning_type in [ImportWarning, SyntaxWarning, DeprecationWarning, UserWarning, RuntimeWarning]:
    warnings.filterwarnings("ignore", category=warning_type)

In [ ]:
# Mount Google Drive, to access the datasets and write your results.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
# ===> SETTINGS TO ADJUST <====

# Your name - refers to the Google Drive folder name
YOUR_NAME = "Matthias"

# city of interest
city = "Timbuktu"
# city = "Davao"

# the simulation's run name
  # => By default, this is set to baseline, representing current conditions.

  # => Alternative scenarios can be done as well. For demonstration purposes, we’ve included
  # a predefined scenario in which all building heights have been tripled.
  # Reach out to the support team in case you want to try something else ...
scenario_name = "baseline"
# scenario_name = "buildingsx3"

# Your own baseline directory - check where you have mounted the ICUC12_Hackathon folder
BASE_DIR = os.path.join('/content/drive/MyDrive/ICUC12_Hackathon')
print('> BASE_DIR: ',BASE_DIR)

In [ ]:
# ===> SETTINGS NOT TO CHANGE <====

# Setting files and directories, based on personal folder, city, and scenario
model = "SOLWEIG"

# Create a link to your personal directory
PERSONAL_FOLDER = os.path.join(BASE_DIR, YOUR_NAME)
print('> PERSONAL_FOLDER: ', PERSONAL_FOLDER)

# Figure folder, to store all your nice visuals
FIGURE_FOLDER = os.path.join(PERSONAL_FOLDER, "figures")
print('> FIGURE_FOLDER: ', FIGURE_FOLDER)

# Create link to shared directory
SHARED_DIR = os.path.join(BASE_DIR, "SHARED_FOLDER")
print('> SHARED_DIR: ',SHARED_DIR)

# Path to shared notebooks
SHARED_FUNCTIONS = os.path.join(SHARED_DIR, "functions")
print(f'> SHARED FUNCTIONS FOLDER: ',SHARED_FUNCTIONS)

# Set MODEL INPUT DATA folder, is the shared data folder
MODEL_INPUT_FOLDER = os.path.join(SHARED_DIR, "data", city, model, scenario_name)
print(f'> {model} INPUT FOLDER: ', MODEL_INPUT_FOLDER)

# Set MODEL OUTPUT folder, under your own personal directory
MODEL_OUTPUT_FOLDER = os.path.join(PERSONAL_FOLDER, city, model, scenario_name)
print(f'> {model} OUTPUT FOLDER: ', MODEL_OUTPUT_FOLDER)

# Set MODEL SOURCE folder, under your own personal directory
MODELS_FOLDER = os.path.join(PERSONAL_FOLDER, "models")
MODEL_SOURCE_FOLDER = os.path.join(MODELS_FOLDER, "umep-core")
print(f'> {model} MODEL SOURCE FOLDER: ', MODEL_SOURCE_FOLDER)

# Paths to initial and updated configsolweig.ini file.
# configsolweig.ini is created below
MODEL_INI_ORIGINAL = os.path.join(MODEL_SOURCE_FOLDER, "umep", "configsolweig.ini")
MODEL_INI_UPDATED = os.path.join(MODEL_OUTPUT_FOLDER, "configsolweig.ini")
print(f'> {model} INI UPDATED: ', MODEL_INI_UPDATED)

# Create some folders that might not exist
for folder in [
    MODEL_OUTPUT_FOLDER,
    MODELS_FOLDER,
    FIGURE_FOLDER
    ]:
  if not os.path.exists(folder):
    os.makedirs(folder)


## Some installations and imports...


In [ ]:
# Install a bunch of packages - with "-q" option to reduce verbose output
%pip install -q import_ipynb pyproj pandas pytz geopandas matplotlib momepy numpy rasterio \
rioxarray scipy shapely tqdm pvlib pyepw pythermalcomfort xarray cftime

In [ ]:
# Import functions from other notebook in Google Colab
import import_ipynb
os.chdir(SHARED_FUNCTIONS)

# import all functions made for this course
import solweig_functions
importlib.reload(solweig_functions)

from solweig_functions import install_umep
from solweig_functions import update_solweigini
from solweig_functions import make_interactive_map_roi
from solweig_functions import make_static_map_roi
from solweig_functions import plot_heatwave_characteristics
from solweig_functions import get_meteo_forcing_day_of_interest
from solweig_functions import plot_walls_and_svf
from solweig_functions import calculate_utci
from solweig_functions import plot_mean_daytime_utci
from solweig_functions import check_position_pois
from solweig_functions import plot_poi_timeseries_utci
from solweig_functions import get_city_info
from solweig_functions import plot_mean_daytime_utci_difference_scenarios
from solweig_functions import plot_poi_timeseries_utci_scenarios

In [ ]:
# Install python version of SOLWEIG
# INFO: https://github.com/UMEP-dev/umep-core

install_umep(MODELS_FOLDER, MODEL_SOURCE_FOLDER)

In [ ]:
# Based on the city, read some info that is city specific and that
# will be used in subsequent steps
cityInfo = get_city_info(city)
print(cityInfo)

# Let's get started

## Context

* what is our area of interest?
* what are the characteristics of the selected heatwave event?
* how is the area represented in the model?

In [ ]:
# Make interactive map of how SOLWEIG sees the underlying surface

m = make_interactive_map_roi(MODEL_INPUT_FOLDER)
m

In [ ]:
# Same as above, but as a static map

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_plot_roi.png")
make_static_map_roi(MODEL_INPUT_FOLDER, figname)

In [ ]:
# How does the meteorology looks like during the selected heat-wave?

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_plot_heatwave_characteristics.png")
plot_heatwave_characteristics(MODEL_INPUT_FOLDER, figname)

## Calculate UTCI

* Run SOLWEIG to get Tmrt
* Get UTCI by combining Tmrt with Tair, U, RH

<br>
Note: for simplicity, Tair, U, and RH are taken from the forcing, instead of using other tools to get more details on their spatially varying characteristics.

In [ ]:
# For the sake of efficiency, we will simulate only one day out of the 7-day heatwave event.
# We can select the day, using the code below

get_meteo_forcing_day_of_interest(MODEL_INPUT_FOLDER, cityInfo['day_of_interest'])

In [ ]:
# Update the solweigconfig.ini files for your case
config = update_solweigini(
    MODEL_SOURCE_FOLDER,
    MODEL_INI_ORIGINAL,
    MODEL_INI_UPDATED,
    MODEL_INPUT_FOLDER,
    MODEL_OUTPUT_FOLDER,
    cityInfo)

In [ ]:
## Prepare the WALL HEIGHT and ASPECT rasters - takes a bit of time
from umep import (
    wall_heightaspect_algorithm,
    skyviewfactor_algorithm,
)

# Look into this, as we need to have the flexibility for scenarios as well ...
if not os.path.exists(config['DEFAULT']['working_dir']):
    os.makedirs(config['DEFAULT']['working_dir'])
    print(f"Created folder: {config['DEFAULT']['working_dir']}")

# wall info for SOLWEIG (height and aspect)
WALL_PATH = os.path.dirname(config['DEFAULT']['filepath_wh'])
if not os.path.exists(WALL_PATH):
    wall_heightaspect_algorithm.generate_wall_hts(
        dsm_path=config['DEFAULT']['filepath_dsm'],
        bbox=None,
        out_dir=WALL_PATH,
        wall_limit=2.0,
    )
else:
    print(f"> Wall properties already available: {WALL_PATH}")


In [ ]:
# Prepare the SKY VIEW FACTORs - takes a bit of time ...

# setup parameters
TRANS_VEG = 3  # Transmissivity trees [DEFAULT]
TRUNK_RATIO = 0.25 # Percent of tree height that is trunk [DEFAULT]

# Get SVF properties
SVF_PATH = os.path.dirname(config['DEFAULT']['input_svf'])
if not os.path.exists(SVF_PATH):
    skyviewfactor_algorithm.generate_svf(
        dsm_path=config['DEFAULT']['filepath_dsm'],
        bbox=None,
        out_dir=SVF_PATH,
        cdsm_path=config['DEFAULT']['filepath_cdsm'],
        trans_veg=TRANS_VEG,
        trunk_ratio=TRUNK_RATIO,
    )
else:
    print(f"> SVF properties already available: {SVF_PATH}")

In [ ]:
# Show the created wall heights/aspects and SVF on a map

filepath_wh = config['DEFAULT']['filepath_wh']
filepath_wa = config['DEFAULT']['filepath_wa']
svf_total_path = os.path.join(MODEL_OUTPUT_FOLDER, "svf", "svf_total.tif")

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_plot_walls_and_svf.png")
plot_walls_and_svf(filepath_wh, filepath_wa, svf_total_path, figname)

In [ ]:
# Calculate Tmrt using SOLWEIG - this can take a bit of time
# Skip step if expected .geotif files already in drive ...

dt = datetime.strptime(cityInfo['day_of_interest'], "%Y-%m-%d")
date_formatted = f"{dt.year}_{dt.timetuple().tm_yday}"
matching_files = glob(os.path.join(MODEL_OUTPUT_FOLDER, f"Tmrt_{date_formatted}*"))

if len(matching_files) == 0:
    print("> No matching output files found, start computing")
    from umep.functions.SOLWEIGpython import Solweig_run as sr
    sr.solweig_run(MODEL_INI_UPDATED, feedback=None)
else:
    print("> Output files aleary available:")
    for f in matching_files:
      print(f)

In [ ]:
# Now, calculate UTCI using Tmrt, U, Tair, and RH, the latter three taken from the forcing (for simplicity)

ds_utci = calculate_utci(config)

## Explore the results

In [ ]:
# Average Tmrt during hours of interest

# Default hours of interest are daytime hours
print("Hours of interest:", cityInfo['hours_of_interest'])

# Make the plot
filepath_utci = os.path.join(config['DEFAULT']['output_dir'], "utci.nc")
figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_plot_mean_daytime_utci.png")

plot_mean_daytime_utci(config, filepath_utci, cityInfo['hours_of_interest'], figname)

In [ ]:
# Get UTCI timeseries for points of interest
# Provide coordinates of points via lat lon values, eg. from Google Maps

# Google Maps link school Davao: https://maps.app.goo.gl/sqxHHbBRLho3zrUi6
# Google Maps link school Timbuktu: https://maps.app.goo.gl/uFbjYPdhFfL1tG2U8

# The coordinates below are examples for the school in Davao
# Feel free to adjust these to explore other locations in the region of interest
if city == "Davao":
  poi1 = [7.088339, 125.623996]  # Northern courtyard, treed
  poi2 = [7.087893, 125.624275]  # Southern courtyard, open
  poi3 = [7.087547, 125.623802]  # Roadside Western entrance, open

# The coordinates below are examples for the school in Timbuktu
elif city == "Timbuktu":
  poi1 = [16.773747, -2.997686]  # Northern courtyard, half-open
  poi2 = [16.773499, -2.997724]  # Southern courtyard, open
  poi3 = [16.773515, -2.997911]  # south-west corner, under large tree

pois = [poi1, poi2, poi3]

In [ ]:
# Check the position of these points of interest on a map
# Adjust above if needed ...

m = check_position_pois(pois)
m

In [ ]:
# Now plot UTCI timeseries for choosen points of interest

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario_name}_plot_poi_timeseries_utci.png")
plot_poi_timeseries_utci(config, filepath_utci, pois, figname)

# What if ...?

The scripts below allow you to compare your scenario against the baseline and assess the impact of such changes on thermal comfort.

In order to do so, utci maps for both scenario_name = "baseline" and scenario_name = "XXX" are required!

In [ ]:
# Average Tmrt during hours of interest

# Default hours of interest are daytime hours
print("Hours of interest:", cityInfo['hours_of_interest'])

# Make the plot
scenario1 = "baseline"
scenario2 = "buildingsx3"

filepath_utci_base = os.path.join(PERSONAL_FOLDER, city, model, scenario1, "utci.nc")
filepath_utci_scen = os.path.join(PERSONAL_FOLDER, city, model, scenario2, "utci.nc")

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}_{scenario1}_{scenario2}_plot_mean_daytime_utci.png")

plot_mean_daytime_utci_difference_scenarios(config, filepath_utci_base, filepath_utci_scen, cityInfo['hours_of_interest'], figname)

In [ ]:
# Plot the timeseries for the points of interest, for both model simulations.

figname = os.path.join(FIGURE_FOLDER, f"{model}_{city}__{scenario1}_{scenario2}_plot_poi_timeseries_utci.png")
plot_poi_timeseries_utci_scenarios(config, filepath_utci_base, filepath_utci_scen, pois, figname)